# Rydberg Blockade Radius

In this notebook we probe the values of the interatomic distance for which we obtain a strong Rydberg Blockade effect.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser.devices import Chadoq2
from pulser.simulation import Simulation
from pulser.waveforms import BlackmanWaveform,ConstantWaveform

In [ ]:
duration = 200
pi_pulse = Pulse(BlackmanWaveform(duration,np.pi/(1000)),ConstantWaveform(duration,0),0)
pi_pulse.draw()

In [ ]:
res = []
N = 6
distances = np.linspace(6,12,N)

In [ ]:
for i,R in enumerate(distances):
    print(f'{N-i} remaining...')
    # Atom Register and Device
    reg = Register.rectangle(1,2,spacing=R, prefix='atom')
    device = Chadoq2(reg) 
    #reg.draw()

    # Pulse Sequence
    seq = Sequence(device)
    seq.declare_channel('ryd','rydberg_local','atom0')
    seq.declare_channel('ryd2','rydberg_local2','atom1')
    seq.add(pi_pulse,'ryd')
    seq.align('ryd','ryd2')
    seq.add(pi_pulse,'ryd2')
    #seq.draw()

    sim = Simulation(seq)
    obs = qutip.tensor([qutip.qeye(2),qutip.basis(2,0)*qutip.basis(2,0).dag()])
    sim.run(obs_list=[obs])
    res.append(sim.output.expect[0])

In [ ]:
for i,R in enumerate(distances):
    plt.plot(res[i], label=f'Interatomic Distance R={R}')
    plt.legend()

We conclude that an interatomic distance between $6$ and $8$ $\mu m$ can give a reasonable Rydberg Blockade radius.